<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [11]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# '일시' 컬럼에서 '월', '일' 열 추가
data['월'] = data['일시'].dt.month
data['일'] = data['일시'].dt.day

# 월, 일별 중앙값 계산
monthly_means = data.groupby('일').transform('mean')
daily_means = data.groupby('일').transform('mean')

<ipython-input-11-45e381c45d0e>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_means = data.groupby('일').transform('mean')
<ipython-input-11-45e381c45d0e>:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily_means = data.groupby('일').transform('mean')


In [5]:
from datetime import datetime

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 29일에 해당하는 16개 행
# 23011 - 16 = 22995

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22992 non-null  float64       
 2   최저기온    22992 non-null  float64       
 3   일교차     22991 non-null  float64       
 4   강수량     9142 non-null   float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22991 non-null  float64       
 7   일조합     22877 non-null  float64       
 8   일사합     18137 non-null  float64       
 9   일조율     22630 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  월       22995 non-null  int64         
 12  일       22995 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(2)
memory usage: 2.5 MB
None


윤년 데이터 빼고 365로 나눠서 모델에 적용시키기

In [12]:
# 결측값 대체
data.fillna(daily_means, inplace=True)

for column in data.columns:
    if data[column].isnull().any():  # 결측값이 있는 경우에만 처리
        data[column+'_중앙값'] = data.apply(lambda row: daily_means.loc[row['일'], column], axis=1)
        data[column].fillna(data[column+'_중앙값'], inplace=True)
        data.drop(column+'_중앙값', axis=1, inplace=True)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22995 non-null  float64       
 2   최저기온    22995 non-null  float64       
 3   일교차     22995 non-null  float64       
 4   강수량     22995 non-null  float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22995 non-null  float64       
 7   일조합     22995 non-null  float64       
 8   일사합     22995 non-null  float64       
 9   일조율     22995 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  월       22995 non-null  int64         
 12  일       22995 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(2)
memory usage: 2.5 MB
None


In [13]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]

target = data['평균기온']

In [14]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(train, target)

pred_xgb = xgb.predict(train)

from sklearn.metrics import mean_squared_error
xgb_mae = mean_squared_error(target, pred_xgb)
print(xgb_mae)

submission['평균기온'] = pred_xgb[:358]
print(submission.head(10))

0.1515267938143393
           일시      평균기온
0  2023-01-01 -2.052494
1  2023-01-02 -2.176404
2  2023-01-03  3.681327
3  2023-01-04  7.008972
4  2023-01-05 -3.859568
5  2023-01-06 -5.391320
6  2023-01-07  1.050129
7  2023-01-08  1.856982
8  2023-01-09  2.364705
9  2023-01-10  1.844559


In [15]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train, target)

pred_rf = rf.predict(train)

from sklearn.metrics import mean_squared_error
rf_mae = mean_squared_error(target, pred_rf)
print(rf_mae)

submission['평균기온'] = pred_rf[:358]
print(submission.head(10))

0.03883357903892152
           일시   평균기온
0  2023-01-01 -1.666
1  2023-01-02 -2.138
2  2023-01-03  3.773
3  2023-01-04  6.968
4  2023-01-05 -4.058
5  2023-01-06 -5.323
6  2023-01-07  1.250
7  2023-01-08  1.817
8  2023-01-09  2.289
9  2023-01-10  1.988


윤년을 뺀게 점수에 영향이 있는지 확인했는데 다행히 빼는게 점수가 더 높게 나왔다